# Create synthetic data with netpyne

In [ ]:
%matplotlib inline
from netpyne import specs, sim
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np

In [ ]:
# Network parameters
netParams = specs.NetParams()  # object of class NetParams to store the network parameters


## Cell parameters
netParams.cellParams['pyr'] = {
    'secs': {
        'soma': {
            'geom': {
                'diam': 18.8,   
                'L': 18.8, 
                'Ra': 123.0},
            'mechs': {
                'hh': {
                    'gnabar': 0.12, 
                    'gkbar': 0.036, 
                    'gl': 0.0003, 
                    'el': -70}
            },
            'ions':{
                'k': {
                    'e': -85,
                }
                
            }
        }
    }
}                


## Population parameters
netParams.popParams['E'] = {
    'cellType': 'pyr', 
    'numCells': 40}

netParams.popParams['I'] = {
    'cellType': 'pyr', 
    'numCells': 10}


## Synaptic mechanism parameters
netParams.synMechParams['exc'] = {
    'mod': 'Exp2Syn', 
    'tau1': 0.1, 
    'tau2': 5.0, 
    'e': 0}  # excitatory synaptic mechanism

netParams.synMechParams['inh'] = {
    'mod': 'Exp2Syn', 
    'tau1': 0.1, 
    'tau2': 5.0, 
    'e': -70}  # inhibitory synaptic mechanism


# Stimulation parameters
netParams.stimSourceParams['bkg'] = {
    'type': 'NetStim', 
    'rate': 50, 
    'noise': 0.5}
    
netParams.stimTargetParams['bkg->E'] = {
    'source': 'bkg', 
    'conds': {'pop': 'E'}, 
    'weight': 0.01, 
    'delay': 5, 
    'synMech': 'exc'}


## Connectivity rules
netParams.connParams['E->I'] = {    #  S -> I label
    'preConds': {'pop': 'E'},       # conditions of presyn cells
    'postConds': {'pop': 'I'},      # conditions of postsyn cells
    'divergence': 5,               # probability of connection
    'weight': 0.01,                 # synaptic weight
    'delay': 5,                     # transmission delay (ms)
    'synMech': 'exc'}               # synaptic mechanism

netParams.connParams['I->E'] = {    #  I -> S label
    'preConds': {'pop': 'I'},       # conditions of presyn cells
    'postConds': {'pop': 'E'},      # conditions of postsyn cells
    'probability': 0.7,               # probability of connection
    'weight': 0.02,                 # synaptic weight
    'delay': 5,                     # transmission delay (ms)
    'synMech': 'inh'}               # synaptic mechanism


# Simulation options
simConfig = specs.SimConfig()       # object of class SimConfig to store simulation configuration

simConfig.duration = 25*1e3          # Duration of the simulation, in ms
simConfig.dt = 0.01                # Internal integration timestep to use
simConfig.verbose = False           # Show detailed messages
simConfig.recordTraces = {'V_soma':{'sec':'soma','loc':0.5,'var':'v'}}  # Dict with traces to record
simConfig.recordStep = 5        # Step size in ms to save data (eg. V traces, LFP, etc)
simConfig.filename = 'tut_oscillation'  # Set file output name
simConfig.saveJson = False

simConfig.recordLFP = [[50, 50, 50]]
simConfig.recordDipole = True

In [ ]:
qtd_sample = 50
all_eeg = []
targets = []
for param_k in tqdm([-55, -85]):
    for i in tqdm(range(qtd_sample)):
        # set reversal potential
        netParams.cellParams['pyr']["secs"]["soma"]["ions"]["k"]["e"] = param_k + np.random.randn()*0.1
        # set rate and noise
        netParams.stimSourceParams['bkg']['rate'] = 50+np.random.randn()*0.1
        netParams.stimSourceParams['bkg']['noise'] = 0.5+np.random.randn()*0.01
        # create network and run simulation
        sim.createSimulateAnalyze(netParams = netParams, simConfig = simConfig)
        array, closest_elec_idx = sim.analysis.plotEEG()
        # we throw away the first 400 points
        save_array = array[closest_elec_idx,400:]
        # save data and target
        all_eeg.append(save_array)
        targets.append(param_k)

In [ ]:
df = pd.DataFrame(all_eeg)
df["target"]=targets

In [ ]:
df.to_csv("eeg_dataset.csv")

## Split data

In [ ]:
file_path = "eeg_dataset.csv"
df = pd.read_csv(file_path)

In [ ]:
def split_df(df):
    
    new_healthy_data = []
    new_healthy_targets = []
    
    new_not_healthy_data = []
    new_not_healthy_targets = []
    
    df.drop(['Unnamed: 0'], inplace=True, axis=1)
    
    df_healthy = df.iloc[:50]
    df_not_healthy = df.iloc[50:]
    
    df_healthy.drop(["target"], axis=1, inplace=True)
    df_not_healthy.drop(["target"], axis=1, inplace=True)
    
    for i in df_healthy.to_numpy():
        new_healthy_data+=np.split(i, 23)
    healthy_targets = ["-55" for i in range(len(new_healthy_data))]

    
    for i in df_not_healthy.to_numpy():
        new_not_healthy_data+=np.split(i, 23)
    not_healthy_targets = ["-85" for i in range(len(new_not_healthy_data))]
        
    data = new_healthy_data+new_not_healthy_data
    target = healthy_targets+not_healthy_targets
    
    final_df = pd.DataFrame(data)
    final_df["target"] = target
        
    return final_df

In [ ]:
final_df = split_df(df)

In [ ]:
final_df.to_csv("eeg_dataset_splited_3.csv")

## Join the simulated data

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
df_1 = pd.read_csv("eeg_dataset_splited_1.csv")
df_2 = pd.read_csv("eeg_dataset_splited_2.csv")
df_3 = pd.read_csv("eeg_dataset_splited_3.csv")

In [ ]:
df_dataset = pd.concat([df_1, df_2, df_3])
df_dataset.sort_values(["target"], inplace=True)

In [ ]:
df_dataset.to_csv("eeg_netpyne_dataset.csv")

## Split train and test data

In [ ]:
df = pd.read_csv("dataset/eeg_netpyne_dataset.csv")
msk = np.random.rand(len(df)) < 0.8

train = df[msk]
test = df[~msk]

In [ ]:
train.target.value_counts()

In [ ]:
test.target.value_counts()

In [ ]:
train.to_csv("dataset/train_eeg_netpyne_dataset.csv")
test.to_csv("dataset/test_eeg_netpyne_dataset.csv")